In [48]:
import importlib
from typing import List, Tuple
import pandas as pd
import numpy as np

from constants import *
import utils
importlib.reload(utils)

<module 'utils' from '/mnt/cb8c2a81-32e7-40bc-af74-0c1386cda683/programowanie/pipelon/Classifier-Builder/lol_classifier/utils.py'>

In [34]:
from glob import glob
import os

datasets = glob(os.path.join(DATASET_PATH, f"*/{DATASET_POSTFIX}"))
print(f"Found {len(datasets)} datasets")

Found 10 datasets


In [35]:
utils.clear_directories([TRAINING_DATA_PATH, TESTING_DATA_PATH])
for path in datasets:
  filename = path.split('/')[-2]

  df = utils.read_eeg_data(path)
  df.drop(df[df.iloc[:, -1] == 'd'].index, inplace=True) # drop flashes
  df.iloc[:SAMPLING_RATE*3, -1]= 'neutral'

  markers = df.iloc[:, -1]
  print(f"From {filename} with shape:{df.shape} and markers:{len(markers[markers != '0'])} kills:{len(markers[markers == 'kill'])}, deaths:{len(markers[markers == 'death'])}")

  prev_marker_and_countdown = ['0', 0]
  df.iloc[:, -1] = markers.apply(lambda x: utils.propagate_events(x, prev_marker_and_countdown))
  
  df.drop(df[markers == '0'].index, inplace=True)  
  df = df.sample(frac=1, random_state=42).reset_index(drop=True)
  utils.split_and_save(df, f"{filename}.csv")
  

From lol_2024-08-04T21-10-29 with shape:(41331, 41) and markers:82 kills:0, deaths:7
From lol_2024-08-08T14-29-23 with shape:(61232, 41) and markers:84 kills:0, deaths:9
From lol_2024-08-08T16-38-51 with shape:(49694, 41) and markers:95 kills:10, deaths:10
From lol_2024-08-08T19-40-51 with shape:(44081, 41) and markers:90 kills:8, deaths:7
From lol_2024-08-08T20-58-04 with shape:(30700, 41) and markers:86 kills:3, deaths:8
From lol_2024-08-08T21-44-59 with shape:(29346, 41) and markers:83 kills:2, deaths:6
From lol_2024-08-09T14-47-50 with shape:(54396, 41) and markers:91 kills:8, deaths:8
From lol_2024-08-12T11-51-55 with shape:(49782, 41) and markers:84 kills:5, deaths:4
From lol_2024-08-12T15-18-29 with shape:(53692, 41) and markers:84 kills:0, deaths:9
From lol_2024-08-12T17-29-12 with shape:(42603, 41) and markers:87 kills:8, deaths:4


In [36]:
train, test = utils.load_datasets()
train.shape, test.shape 

Number of kills=2537; deaths=4239; neutral=1190


((7966, 41), (1997, 41))

In [37]:
train[np.isnan(train).any(axis=1)]


array([], shape=(0, 41), dtype=float64)

In [52]:
from sklearn.svm import SVC

X_train, y_train = train[:, :-1], train[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]

svm = SVC(kernel='poly', degree=3, C=1, random_state=42, decision_function_shape='ovr', probability=True)
svm.fit(X_train, y_train)

SVC(C=1, kernel='poly', probability=True, random_state=42)

In [53]:
print('accuracy:', svm.score(X_test, y_test))

from sklearn.metrics import classification_report, confusion_matrix
y_pred = svm.predict(X_test)

utils.present_confusion_matrix(confusion_matrix(y_test, y_pred))
utils.present_metrics(classification_report(y_test, y_pred, target_names=STATES, output_dict=True))

accuracy: 0.9053580370555834
Confusion Matrix:
+-----------------+-----------------+------------------+--------------------+
|                 | Predicted: kill | Predicted: death | Predicted: neutral |
+-----------------+-----------------+------------------+--------------------+
|  Actual: kill   |       547       |        89        |         0          |
|  Actual: death  |       90        |       964        |         7          |
| Actual: neutral |        1        |        2         |        297         |
+-----------------+-----------------+------------------+--------------------+

Classification Report:
+--------------+-----------+--------+----------+---------+
|              | precision | recall | f1-score | support |
+--------------+-----------+--------+----------+---------+
|     kill     |   0.857   |  0.86  |  0.859   |  636.0  |
|    death     |   0.914   | 0.909  |  0.911   | 1061.0  |
|   neutral    |   0.977   |  0.99  |  0.983   |  300.0  |
|   accuracy   |   0.905   | 

In [55]:
from skl2onnx import to_onnx
onx = to_onnx(svm, X_train[:1].astype(np.float32), target_opset=12)
with open("output/svm-without-preprocessing-90p.onnx", "wb") as f:
    f.write(onx.SerializeToString())
